# Basic usage of the MICADO spectroscopy mode in ScopeSim

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"><br></div>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

## Set up ScopeSim and MICADO

The following ``import`` statements are needed to run ScopeSim.

The ``pkgs_path`` value can be changed to point to the folder where you are keeping the instrument packages. If you are using the default settings, then ScopeSim will download these packages to your working directory (where this notebook is) and move them into the ``inst_pkgs/`` subdirectory.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy import units as u

import scopesim as sim
from scopesim.source import source_templates as src_tp
import scopesim_templates as sim_tp
%matplotlib inline

# sim.rc.__config__["!SIM.file.local_packages_path"] = "inst_simulations/inst_pkgs"

If you have not done so already, please download the relevant instrument packages using the following code in a new cell:

    sim.download_packages(["Armazones", "ELT", "MORFEO", "MICADO"])
    
Alternatively, if we would like to keep the instrument packages in a separate directory, we can set the following config value:

    sim.rc.__config__["!SIM.file.local_packages_path"] = "path/to/packages"


## Basic on-sky source to observe

Some basic on-sky source examples to pick from. Note, that sources can be added together with the `+` operator.

In [ ]:
# src = sim_tp.empty_sky()
src = sim_tp.star("K", 10)
# src = src_tp.uniform_source(extent=60)
# src = src + src_tp.uniform_illumination(xs=[-8, 8], ys=[-0.03, 0.03], pixel_scale=0.004, flux=10*u.mag)
# src = sim_tp.extragalactic.galaxies.spiral_two_component(extent=16*u.arcsec, fluxes=(15, 15)*u.mag)

## Part 1: Set up MICADO for spectroscopy with a 3000x48 slit and the Spec_IJ filter

We start off by importing all the configuration data with ``UserCommands``. 
Here we also specify which modes are to be used. 
To start with, we will use the 15 arcsec slit in the ``Spec_IJ`` filter.

The ``set_modes`` parameter takes a list of two mode keywords: ``[<AO mode>, <slit>]``. 
- AO modes affect total system transmisison and which PSF file is used.
  - Possible values are ``SCAO``, ``MCAO``
- The slit modes difine the slit dimensions (and add the grating losses to the total transmission).
  The mode name reflects the dimensions of the slit in  [mas]:
  - Possible values are ``SPEC_3000x16``, ``SPEC_3000x48``, ``SPEC_15000x20``

Some possible combinations would be:

    set_modes=["SCAO", "SPEC_3000x16"]
    set_modes=["MCAO", "SPEC_3000x48"]
    set_modes=["SCAO", "SPEC_15000x20"]

Possible combinations of filter and slit are:

    Spec_IJ, SPEC_3000x48
    Spec_IJ, SPEC_3000x16
    Spec_J, SPEC_15000x20
    Spec_HK, SPEC_3000x48
    Spec_HK, SPEC_1500020


The filter is set by default to the ``Spec_HK`` filter, located in filter wheel 1.

To change the filter, set it to the ``"Spec_IJ"`` or ``"J"`` filter by useing the setting: ``cmd["!OBS.filter_name_fw1"] = "Spec_IJ"``

In [ ]:
AO_MODE = "SCAO"
SLIT = "SPEC_3000x48"
FILT = "Spec_IJ"

cmd = sim.UserCommands(use_instrument="MICADO", set_modes=[AO_MODE, SLIT])

# we can swap to imaging mode just to check what the field looks like
# cmd = sim.UserCommands(use_instrument="MICADO", set_modes=["SCAO", "IMG_4mas"])

cmd["!OBS.filter_name_fw1"] = FILT
cmd["!OBS.filter_name_fw2"] = "open"


Select an appropriate spectral resolution based on filter and slit width

In [ ]:
min_wavelen = {
    "Spec_IJ": 0.82,
    "Spec_HK": 1.49, 
    "J": 1.16,
    } # in microns

R = {
    "SPEC_3000x16": 10000,
    "SPEC_3000x48": 3300,
    "SPEC_15000x20": 8000,
}

def spec_res(slit, filter):
    return min_wavelen[filter] / R[slit]

This notebook runs for a long time and requires more than 30 GB of RAM. It is possible to speed up the computation and reduce the memory consumption (at the cost of lower accuracy) by specifying the spectral bin width and disabling PSF interpolation. Uncomment the relevant lines below if you prefer a faster but less accurate simulation:

In [ ]:
cmd["!SIM.spectral.spectral_bin_width"] = spec_res(SLIT, FILT)

# settings to speed up the simulation
# cmd["!SIM.spectral.spectral_bin_width"] = 5e-4
cmd["!OBS.interp_psf"] = False

print(f"Spectral bin width set to {cmd['!SIM.spectral.spectral_bin_width']:.2e} microns")

# If not usign the full detector (see below), set the size of the window to simulate
cmd["!DET.width"] = 4096
cmd["!DET.height"] = 4096

# change the psf file if needed (e.g. for a larger extent version)
# cmd["!RO.psf_file"] = "inst_simulations/inst_pkgs/MICADO/SCAO_ConstPSF_5off_1024.fits"

In [ ]:
micado = sim.OpticalTrain(cmd)

For testing purposes it maybe useful to only simulate a certain sub-window on the focal plane. 

If we want to simulate the full detector array, we need to exclude the ``detector_window`` effect, and include the ``full_detector_array`` effect. 

Naturally the execution time is dependent on whether we want to simulate only a small fraction of the focal plane, or the full thing.

In [ ]:
USE_FULL_DETECTOR = False
micado["full_detector_array"].include = USE_FULL_DETECTOR
micado["detector_window"].include = not USE_FULL_DETECTOR

To check which instrument configuration is currently used, we can print the table of effects with ``micado.effects``

In [ ]:
micado.effects.pprint_all()

## Observe the on-sky source

Running the simulation involves first ``observe()``ing the object, then ``readout()``ing the detectors.

    micado.observe(src)
    micado.readout()
    
``observe()`` does not produce any output, although it is possible to access (and save) the intermediate focal plane image.

``readout()`` returns a list of ``fits.HDUList`` object to the ipython session. 

By providing the ``filename`` keyword, the result can also be saved to disk: ``micado.readout(filename="spec_temp.fits")``

In [ ]:
# Warning, this takes about 1 minute per detector to complete
# (with PSF interpolation turned off)
micado.observe(src)

### Checking the intermediate output

We can check the intermediate (noise-less) focal plane image by looking at the ``micado.image_planes[0]`` ``ImageHDU`` object:

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(micado.image_planes[0].data, norm=LogNorm(), origin="lower")
plt.colorbar()

### Performing the detector readout



Change ``dit`` and ``ndit`` to whatever we want the exposure time to be:

In [ ]:
hdul = micado.readout(dit=60, ndit=1)

### Checking the final output

Now we plot the noisy detector readouts

In [ ]:
# detector layout definition in the form subplot index: hdu index
hdu_layout = {
    1: 1,
    2: 2,
    3: 3,
    4: 6,
    5: 5,
    6: 4,
    7: 7,
    8: 8,
    9: 9,
}

plt.figure(figsize=(15,15))
if USE_FULL_DETECTOR:
    for i in range(1,10):
        plt.subplot(3,3,i)
        plt.imshow(hdul[0][hdu_layout[i]].data, norm=LogNorm(), origin="lower")
else:
    plt.imshow(hdul[0][1].data, norm=LogNorm(), origin="lower")
    plt.colorbar()

## Part 2: Observe the on-sky source with a 15000x20 slit in the Spec_HK filter

To change modes, we need to create a new model of MICADO.

The filter is set by default to the ``Spec_HK`` filter, located in filter wheel 1.

In [ ]:
FILT = "Spec_HK"
SLIT = "SPEC_15000x20"

cmd = sim.UserCommands(use_instrument="MICADO", set_modes=["SCAO", SLIT])
cmd["!OBS.filter_name_fw1"] = FILT
cmd["!OBS.filter_name_fw2"] = "open"

cmd["!SIM.spectral.spectral_bin_width"] = spec_res(SLIT, FILT)

# un-comment the following lines to speed up the simulation at the cost of accuracy
# cmd["!SIM.spectral.spectral_bin_width"] = spec_res(SLIT, FILT)
cmd["!OBS.interp_psf"] = False

# If not usign the full detector (see below), set the size of the window to simulate
cmd["!DET.width"] = 4096
cmd["!DET.height"] = 4096

# change the psf file if needed (e.g. for a larger extent version)
# cmd["!RO.psf_file"] = "inst_simulations/inst_pkgs/MICADO/SCAO_ConstPSF_5off_1024.fits"

micado = sim.OpticalTrain(cmd)

USE_FULL_DETECTOR = False
micado["full_detector_array"].include = USE_FULL_DETECTOR
micado["detector_window"].include = not USE_FULL_DETECTOR

In [ ]:
# Warning, simulating the long slit takes about 5 minutes per detector to complete
# (with psf interpolation turned off)
micado.observe(src)

### Checking the intermediate output

We can check the intermediate (noise-less) focal plane image by looking at the ``micado.image_planes[0]`` ``ImageHDU`` object:

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(micado.image_planes[0].data, norm=LogNorm(), origin="lower")

### Performing the readout

In [ ]:
hdul = micado.readout(dit=60, ndit=10)

### Checking the final output

Now we plot the noisy detector readouts

In [ ]:
plt.figure(figsize=(15,15))
if USE_FULL_DETECTOR:
    for i in range(1,10):
        plt.subplot(3,3,i)

        im = hdul[0][hdu_layout[i]].data
        plt.imshow(im, norm=LogNorm(vmin=0.99*np.median(im), vmax=1.1*np.median(im)), origin="lower")
else:
    im = hdul[0][1].data
    plt.imshow(im, norm=LogNorm(vmin=0.99*np.median(im), vmax=1.1*np.median(im)), origin="lower")

## Saving to disk

All the returned objects are from the ``astropy.io.fits`` library, and can therefore be saved to disk in the usual manner:

In [ ]:
# micado.image_planes[0].hdu.writeto("intermediate_data.fits", overwrite=True)
# hdul[0].writeto("detector_readout_data.fits", overwrite=True)